imports

In [292]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchtext
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

from tqdm.auto import tqdm

upload datasets

In [293]:
def str_text_to_list(value):
    list_values = value.strip('[]').split(', ')
    cleaned_list_values = [item[1:-1] for item in list_values]
    return cleaned_list_values

In [294]:
dataset_clean = pd.read_csv("../datasets/tonetags_dataset_tumblr_clean.csv", converters={"text": str_text_to_list})

In [295]:
dataset_clean_corrected = pd.read_csv("../datasets/tonetags_dataset_tumblr_clean_corrected_text.csv", converters={"text": str_text_to_list})

In [296]:
dataset_wsd_1 = pd.read_csv("../datasets/tonetags_wsd_1.csv", index_col=0, converters={"text": str_text_to_list})

In [297]:
def str_context_to_list(value):
    list_values = value.strip('[]').split(' ')
    cleaned_list_values = []
    for item in list_values:
        item.strip()
        # if '\n' in item:
        #     item.replace('\n', '')
        if item != "":
            cleaned_list_values.append(float(item))
    return cleaned_list_values

In [298]:
dataset_wsd_2 = pd.read_csv("../datasets/tonetags_wsd_2.csv", index_col=0, converters={"text": str_text_to_list, 'context': str_context_to_list})

encode labels

In [299]:
labels = dataset_wsd_1.tags.unique().tolist()

dataset_wsd_1.tags = dataset_wsd_1.tags.apply(labels.index)

dataset_clean.tags = dataset_clean.tags.apply(labels.index)
dataset_clean_corrected.tags =dataset_clean_corrected.tags.apply(labels.index)
dataset_wsd_2.tags = dataset_wsd_2.tags.apply(labels.index)

split data

In [300]:
train_clean, test_clean = train_test_split(dataset_clean, test_size=0.2, shuffle=False)

In [301]:
train_clean_corrected, test_clean_corrected = train_test_split(dataset_clean_corrected, test_size=0.2, shuffle=False)

In [302]:
train_wsd_1, test_wsd_1 = train_test_split(dataset_wsd_1, test_size=0.2, shuffle=False)

In [303]:
train_wsd_2, test_wsd_2 = train_test_split(dataset_wsd_2, test_size=0.2, shuffle=False)

create vocab and class of dataset

In [304]:
glove_6b_50 = torchtext.vocab.GloVe(name='6B', dim=50)

In [305]:
vocab = glove_6b_50.stoi
vocab["<unk>"] = len(vocab)
vocab["<pad>"] = len(vocab)

In [306]:
class myDataset(Dataset):
    def __init__(self, dataset):
        
        max_length = 4096
        
        self.data = []
        for sentence in dataset.text:
            if len(sentence) > max_length:
                continue
            sentence_ids = []
            for token in sentence:
                try:
                    sentence_ids.append(vocab[token])
                except KeyError:
                    sentence_ids.append(vocab["<unk>"])
            self.data.append(sentence_ids)
        self.labels = dataset.tags
        
        self.context = None
        
        if 'context' in dataset.columns:
            self.context = dataset.context

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.context is None:
            return self.data[idx], torch.tensor(self.labels.iloc[idx])
        else:
            return self.data[idx], torch.tensor(self.labels.iloc[idx]), self.context.iloc[idx]

In [307]:
def collate_fn(batch):
    max_length = 4096
    
    data_ids = []
    labels = []
    contexts = []
    
    for dat in batch:
        
        data_ids.append(dat[0])
        labels.append(dat[1])
        
        if len(dat) >= 3:
            contexts.append(dat[2])

    for i in range(len(data_ids)):
        while len(data_ids[i]) < max_length:
            data_ids[i].append(vocab["<pad>"])
    
    return torch.tensor(data_ids), torch.tensor(contexts), torch.tensor(labels)

create datasets

In [308]:
train_dataset_clean, test_dataset_clean = myDataset(train_clean), myDataset(test_clean)

In [309]:
train_dataset_clean_corrected, test_dataset_clean_corrected = myDataset(train_clean_corrected), myDataset(test_clean_corrected)

In [310]:
train_dataset_wsd_1, test_dataset_wsd_1 = myDataset(train_wsd_1), myDataset(test_wsd_1)

In [311]:
train_dataset_wsd_2, test_dataset_wsd_2 = myDataset(train_wsd_2), myDataset(test_wsd_2)

create dataloaders

In [312]:
batch_size = 32

In [313]:
train_dataloader_clean = DataLoader(train_dataset_clean, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

test_dataloader_clean = DataLoader(test_dataset_clean, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [314]:
train_dataloader_clean_corrected = DataLoader(train_dataset_clean_corrected, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

test_dataloader_clean_corrected = DataLoader(test_dataset_clean_corrected, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [315]:
train_dataloader_wsd_1 = DataLoader(train_dataset_wsd_1, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

test_dataloader_wsd_1 = DataLoader(test_dataset_wsd_1, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [316]:
train_dataloader_wsd_2 = DataLoader(train_dataset_wsd_2, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

test_dataloader_wsd_2 = DataLoader(test_dataset_wsd_2, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

tokens embedding

In [317]:
embedding_vector = glove_6b_50.vectors.numpy()
embedding_vector = np.append(embedding_vector, np.zeros(50)).reshape(-1, 50) # vector for unknown value in vocab
embedding_vector = np.append(embedding_vector, np.ones(50)).reshape(-1, 50) # vector for padding value in vocab

In [318]:
embedding_tensor = torch.tensor(embedding_vector, dtype=torch.float)

In [319]:
nn.Embedding.from_pretrained(embedding_tensor, freeze=True)

Embedding(400002, 50)

model

In [320]:
class ToneTagsLSTM(nn.Module):
    def __init__(self, vocab_size, embedding, hidden_dim, output_size, num_layers, dropout):
        super(ToneTagsLSTM, self).__init__()
        # vocab_size = 400002
        # embedding_dim = 50
        # hidden_dim_lstm = 30
        # fc1 = 30 - 25
        # fc2 = 25 - 20
        # fc3 = 20 - 19
        # output_size = 19
        
        self.embedding = embedding
        
        self.lstm = nn.LSTM(self.embedding.embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=True, dropout=dropout, batch_first=True)
        
        self.fc1 = nn.Linear(hidden_dim, 25)
        self.fc2 = nn.Linear(25, 20)
        self.fc3 = nn.Linear(20, output_size)
        # self.out = nn.Softmax(output_size, dim=1)
        

    def forward(self, x):
        
        embedded = self.embedding(x)
        output, (hidden, cell) = self.lstm(embedded)


        # print(hidden[-1].shape)

        fc1_out = self.fc1(hidden[-1])
        
        
        fc2_out = self.fc2(fc1_out)
        out = self.fc3(fc2_out)
        # out = self.out(fc3_out)
        
        return out

parameters

In [321]:
vocab_size = len(vocab) # 4000002
embedding = nn.Embedding.from_pretrained(embedding_tensor, freeze=True)
hidden_dim = 30 # 30
output_size = len(labels) # 19
num_layers = 4
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lr = 5e-4

criterion = nn.CrossEntropyLoss()

accuracy function

In [322]:
def get_accuracy(prediction, label):
    batch_size, _ = prediction.shape
    predicted_classes = prediction.argmax(dim=-1)
    correct_predictions = predicted_classes.eq(label).sum()
    accuracy = correct_predictions / batch_size
    return accuracy

train and test `train_dataloader_wsd_1`

In [323]:
# train_dataloader_wsd_1
# test_dataloader_wsd_1

model_wsd_1 = ToneTagsLSTM(
    vocab_size=vocab_size,
    embedding=embedding,
    hidden_dim=hidden_dim,
    output_size=output_size,
    num_layers=num_layers,
    dropout=dropout
).to(device)

optimizer_wsd_1 = optim.Adam(model_wsd_1.parameters(), lr=lr)

In [324]:
%%time

n_epochs = 20

for ep in range(1, n_epochs + 1):
    
    model_wsd_1.train()
    epoch_losses_train = []
    epoch_accuracies_train = []
    
    
    for tokens, contexts, tags in tqdm(train_dataloader_wsd_1, desc="training..."):
        optimizer_wsd_1.zero_grad()
        
        tokens = tokens.to(device)
        tags = tags.to(device)
        
        predictions = model_wsd_1(tokens)
        
        loss = criterion(predictions, tags)
        accuracy = get_accuracy(predictions, tags)
        
        loss.backward()
        optimizer_wsd_1.step()
        
        epoch_losses_train.append(loss.item())
        epoch_accuracies_train.append(accuracy.item())
        
    print(f'[Train Epoch {ep}] Loss: {np.mean(epoch_losses_train)}, Accuracy: {np.mean(epoch_accuracies_train)}')


    model_wsd_1.eval()
    epoch_losses_test = []
    epoch_accuracies_test = []
    
    with torch.no_grad():
        for tokens, contexts, tags in tqdm(test_dataloader_wsd_1, desc="testing..."):
            optimizer_wsd_1.zero_grad()
    
            tokens = tokens.to(device)
            tags = tags.to(device)
    
            predictions = model_wsd_1(tokens)
    
            loss = criterion(predictions, tags)
            accuracy = get_accuracy(predictions, tags)
    
            epoch_losses_test.append(loss.item())
            epoch_accuracies_test.append(accuracy.item())
    
        print(f'[Test Epoch {ep}] Loss: {np.mean(epoch_losses_test)}, Accuracy: {np.mean(epoch_accuracies_test)}\n')

training...:   0%|          | 0/2113 [00:00<?, ?it/s]

[Train Epoch 1] Loss: 2.379364491073204, Accuracy: 0.21823237103644108


testing...:   0%|          | 0/530 [00:00<?, ?it/s]

[Test Epoch 1] Loss: 6.908232303835311, Accuracy: 0.0026533018867924527


training...:   0%|          | 0/2113 [00:00<?, ?it/s]

...

[Test Epoch 19] Loss: 6.817184558904396, Accuracy: 0.1248820754716981


training...:   0%|          | 0/2113 [00:00<?, ?it/s]

[Train Epoch 20] Loss: 1.6621024412562708, Accuracy: 0.5012866777094179


testing...:   0%|          | 0/530 [00:00<?, ?it/s]

[Test Epoch 20] Loss: 7.059619648951404, Accuracy: 0.12750691282299328

CPU times: total: 1h 20min 45s
Wall time: 1h 20min 28s


train and test `train_dataloader_clean`

In [325]:
# train_dataloader_clean
# test_dataloader_clean

model_clean = ToneTagsLSTM(
    vocab_size=vocab_size,
    embedding=embedding,
    hidden_dim=hidden_dim,
    output_size=output_size,
    num_layers=num_layers,
    dropout=dropout
).to(device)

optimizer_clean = optim.Adam(model_clean.parameters(), lr=lr)

In [326]:
%%time

n_epochs = 20

for ep in range(1, n_epochs + 1):

    model_clean.train()
    epoch_losses_train = []
    epoch_accuracies_train = []


    for tokens, contexts, tags in tqdm(train_dataloader_clean, desc="training..."):
        optimizer_clean.zero_grad()

        tokens = tokens.to(device)
        tags = tags.to(device)

        predictions = model_clean(tokens)

        loss = criterion(predictions, tags)
        accuracy = get_accuracy(predictions, tags)

        loss.backward()
        optimizer_clean.step()

        epoch_losses_train.append(loss.item())
        epoch_accuracies_train.append(accuracy.item())

    print(f'[Train Epoch {ep}] Loss: {np.mean(epoch_losses_train)}, Accuracy: {np.mean(epoch_accuracies_train)}')


    model_clean.eval()
    epoch_losses_test = []
    epoch_accuracies_test = []

    with torch.no_grad():
        for tokens, contexts, tags in tqdm(test_dataloader_clean, desc="testing..."):
            optimizer_clean.zero_grad()

            tokens = tokens.to(device)
            tags = tags.to(device)

            predictions = model_clean(tokens)

            loss = criterion(predictions, tags)
            accuracy = get_accuracy(predictions, tags)

            epoch_losses_test.append(loss.item())
            epoch_accuracies_test.append(accuracy.item())

        print(f'[Test Epoch {ep}] Loss: {np.mean(epoch_losses_test)}, Accuracy: {np.mean(epoch_accuracies_test)}\n')

training...:   0%|          | 0/2132 [00:00<?, ?it/s]

[Train Epoch 1] Loss: 2.243861869005504, Accuracy: 0.2783780878804638


testing...:   0%|          | 0/534 [00:00<?, ?it/s]

[Test Epoch 1] Loss: 6.458495038725464, Accuracy: 0.003920880149812734


training...:   0%|          | 0/2132 [00:00<?, ?it/s]

...

[Test Epoch 19] Loss: 6.97753054550971, Accuracy: 0.1346558988764045


training...:   0%|          | 0/2132 [00:00<?, ?it/s]

[Train Epoch 20] Loss: 1.3589751783611628, Accuracy: 0.5857645403488939


testing...:   0%|          | 0/534 [00:00<?, ?it/s]

[Test Epoch 20] Loss: 6.732687687605955, Accuracy: 0.12265917602996254

CPU times: total: 1h 21min 47s
Wall time: 1h 21min 31s


train and test `train_dataloader_clean_corrected`

In [327]:
# train_dataloader_clean_corrected
# test_dataloader_clean_corrected

model_clean_corrected = ToneTagsLSTM(
    vocab_size=vocab_size,
    embedding=embedding,
    hidden_dim=hidden_dim,
    output_size=output_size,
    num_layers=num_layers,
    dropout=dropout
).to(device)

optimizer_clean_corrected = optim.Adam(model_clean_corrected.parameters(), lr=lr)

In [328]:
%%time

n_epochs = 20

for ep in range(1, n_epochs + 1):

    model_clean_corrected.train()
    epoch_losses_train = []
    epoch_accuracies_train = []


    for tokens, contexts, tags in tqdm(train_dataloader_clean_corrected, desc="training..."):
        optimizer_clean_corrected.zero_grad()

        tokens = tokens.to(device)
        tags = tags.to(device)

        predictions = model_clean_corrected(tokens)

        loss = criterion(predictions, tags)
        accuracy = get_accuracy(predictions, tags)

        loss.backward()
        optimizer_clean_corrected.step()

        epoch_losses_train.append(loss.item())
        epoch_accuracies_train.append(accuracy.item())

    print(f'[Train Epoch {ep}] Loss: {np.mean(epoch_losses_train)}, Accuracy: {np.mean(epoch_accuracies_train)}')


    model_clean_corrected.eval()
    epoch_losses_test = []
    epoch_accuracies_test = []

    with torch.no_grad():
        for tokens, contexts, tags in tqdm(test_dataloader_clean_corrected, desc="testing..."):
            optimizer_clean.zero_grad()

            tokens = tokens.to(device)
            tags = tags.to(device)

            predictions = model_clean_corrected(tokens)

            loss = criterion(predictions, tags)
            accuracy = get_accuracy(predictions, tags)

            epoch_losses_test.append(loss.item())
            epoch_accuracies_test.append(accuracy.item())

        print(f'[Test Epoch {ep}] Loss: {np.mean(epoch_losses_test)}, Accuracy: {np.mean(epoch_accuracies_test)}\n')

training...:   0%|          | 0/2128 [00:00<?, ?it/s]

[Train Epoch 1] Loss: 2.275204872614459, Accuracy: 0.26903195488721804


testing...:   0%|          | 0/532 [00:00<?, ?it/s]

[Test Epoch 1] Loss: 6.545017553451366, Accuracy: 0.04387922932330827


training...:   0%|          | 0/2128 [00:00<?, ?it/s]

...

[Test Epoch 19] Loss: 7.0642170314502, Accuracy: 0.13270285088372857


training...:   0%|          | 0/2128 [00:00<?, ?it/s]

[Train Epoch 20] Loss: 1.3681867054892929, Accuracy: 0.5844249295112782


testing...:   0%|          | 0/532 [00:00<?, ?it/s]

[Test Epoch 20] Loss: 7.046278788631124, Accuracy: 0.1322564223319068

CPU times: total: 1h 20min 58s
Wall time: 1h 20min 47s


train and test `train_dataloader_wsd_2`

In [329]:
# train_dataloader_wsd_2
# test_dataloader_wsd_2

vocab_size = len(vocab) # 4000002
embedding = nn.Embedding.from_pretrained(embedding_tensor, freeze=True)
hidden_dim = 30 # 30
context_dim = 50 # 50
output_size = len(labels) # 19
num_layers = 4
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lr = 5e-4

criterion = nn.CrossEntropyLoss()

In [330]:
class ToneTagsLSTM_wsd_2(nn.Module):
    def __init__(self, vocab_size, embedding, hidden_dim, context_dim, output_size, num_layers, dropout):
        super(ToneTagsLSTM_wsd_2, self).__init__()
        # vocab_size = 400002
        # embedding_dim = 50
        # hidden_dim_lstm = 30
        # fc1 = 30 + 50 - 60
        # fc2 = 60 - 30
        # fc3 = 30 - 19
        # output_size = 19

        self.embedding = embedding

        self.lstm = nn.LSTM(self.embedding.embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=True, dropout=dropout, batch_first=True)

        self.fc1 = nn.Linear(hidden_dim + context_dim, 60)
        self.fc2 = nn.Linear(60, 30)
        self.fc3 = nn.Linear(30, output_size)
        # self.out = nn.Softmax(output_size, dim=1)


    def forward(self, tokens, contexts):

        embedded = self.embedding(tokens)
        output, (hidden, cell) = self.lstm(embedded)

        lstm_out = torch.cat((hidden[-1], contexts), dim=1)

        fc1_out = self.fc1(lstm_out)


        fc2_out = self.fc2(fc1_out)
        out = self.fc3(fc2_out)
        # out = self.out(fc3_out)

        return out

In [331]:
# train_dataloader_wsd_2
# test_dataloader_wsd_2

model_wsd_2 = ToneTagsLSTM_wsd_2(
    vocab_size=vocab_size,
    embedding=embedding,
    hidden_dim=hidden_dim,
    context_dim=context_dim,
    output_size=output_size,
    num_layers=num_layers,
    dropout=dropout
).to(device)

optimizer_wsd_2 = optim.Adam(model_wsd_2.parameters(), lr=lr)

In [332]:
%%time

n_epochs = 20

for ep in range(1, n_epochs + 1):

    model_wsd_2.train()
    epoch_losses_train = []
    epoch_accuracies_train = []


    for tokens, contexts, tags in tqdm(train_dataloader_wsd_2, desc="training..."):
        optimizer_wsd_2.zero_grad()

        tokens = tokens.to(device)
        contexts = contexts.to(device)
        tags = tags.to(device)

        predictions = model_wsd_2(tokens, contexts)

        loss = criterion(predictions, tags)
        accuracy = get_accuracy(predictions, tags)

        loss.backward()
        optimizer_wsd_2.step()

        epoch_losses_train.append(loss.item())
        epoch_accuracies_train.append(accuracy.item())

    print(f'[Train Epoch {ep}] Loss: {np.mean(epoch_losses_train)}, Accuracy: {np.mean(epoch_accuracies_train)}')


    model_wsd_2.eval()
    epoch_losses_test = []
    epoch_accuracies_test = []

    with torch.no_grad():
        for tokens, contexts, tags in tqdm(test_dataloader_wsd_2, desc="testing..."):
            optimizer_wsd_2.zero_grad()

            tokens = tokens.to(device)
            contexts = contexts.to(device)
            tags = tags.to(device)

            predictions = model_wsd_2(tokens, contexts)

            loss = criterion(predictions, tags)
            accuracy = get_accuracy(predictions, tags)

            epoch_losses_test.append(loss.item())
            epoch_accuracies_test.append(accuracy.item())

        print(f'[Test Epoch {ep}] Loss: {np.mean(epoch_losses_test)}, Accuracy: {np.mean(epoch_accuracies_test)}\n')

training...:   0%|          | 0/2132 [00:00<?, ?it/s]

ValueError: expected sequence of length 50 at dim 1 (got 0)